In [3]:
# pip install sgnlp

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Sentic-GCN: Aspect-Based Sentiment Analysis via Affective Knowledge Enhanced Graph Convolutional Networks

In [2]:
from sgnlp.models.sentic_gcn import(
    SenticGCNBertConfig,
    SenticGCNBertModel,
    SenticGCNBertEmbeddingConfig,
    SenticGCNBertEmbeddingModel,
    SenticGCNBertTokenizer,
    SenticGCNBertPreprocessor,
    SenticGCNBertPostprocessor
)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
tokenizer = SenticGCNBertTokenizer.from_pretrained("bert-base-uncased")

config = SenticGCNBertConfig.from_pretrained(
    "https://storage.googleapis.com/sgnlp/models/sentic_gcn/senticgcn_bert/config.json"
)

model = SenticGCNBertModel.from_pretrained(
    "https://storage.googleapis.com/sgnlp/models/sentic_gcn/senticgcn_bert/pytorch_model.bin",
    config=config
)

embed_config = SenticGCNBertEmbeddingConfig.from_pretrained("bert-base-uncased")

embed_model = SenticGCNBertEmbeddingModel.from_pretrained("bert-base-uncased",
    config=embed_config
)

preprocessor = SenticGCNBertPreprocessor(
    tokenizer=tokenizer, embedding_model=embed_model,
    senticnet="https://storage.googleapis.com/sgnlp/models/sentic_gcn/senticnet.pickle",
    device="cpu")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SenticGCNBertTokenizer'.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing SenticGCNBertEmbeddingModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing SenticGCNBertEmbeddingModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SenticGCNBertEmbeddingModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
postprocessor = SenticGCNBertPostprocessor()
# Aspect must be inside the sentence
inputs = [
    {  # Single word aspect
        "aspects": ["Service"],
        "sentence": "Service was average",
    }
]
processed_inputs, processed_indices = preprocessor(inputs)
raw_outputs = model(processed_indices)
post_outputs = postprocessor(processed_inputs=processed_inputs, model_outputs=raw_outputs)

In [15]:
post_outputs[0]['labels']
#predicted positive

[1]

Here I try to use two word aspect.

In [19]:
inputs = [
    {  # Two word aspect
        "aspects": ["service",'Food'],
        "sentence": "Service is very bad. Food is very bad. I will never come back again!",
    }
]
processed_inputs, processed_indices = preprocessor(inputs)
raw_outputs = model(processed_indices)
post_outputs = postprocessor(processed_inputs=processed_inputs, model_outputs=raw_outputs)

In [20]:
post_outputs[0]['labels']
#predicted negative.
# Negative for service, Negative for food.

[-1, -1]

# UFD: Unsupervised Domain Adaptation of a Pretrained Cross-Lingual Language Model

In [9]:
from sgnlp.models.ufd import (
   UFDModelBuilder,
   UFDPreprocessor)
import torch
import torch.nn.functional as F

By default, UFDModelBuilder will include all available pretrained models, to target only specific model set, simply define the source_domains, target_languages and target_domains input arguments. The following shows an example for a single model set for the books source domains, German target language and dvd target domain.

In [24]:
model_builder = UFDModelBuilder()
preprocessor = UFDPreprocessor()
model_groups = model_builder.build_model_group()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /xlm-roberta-large/resolve/main/sentencepiece.bpe.model HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /xlm-roberta-large/resolve/main/special_tokens_map.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLMRobertaTokenizer'. 
The class this function is called from is 'UFDTokenizer'.
DEBUG:urllib3.connectionp

In [26]:
text = ['extremely bad, i hate it']
text_feature = preprocessor(text)
output = model_groups['books_de_dvd'](**text_feature)
logits_probabilities = F.softmax(output.logits, dim=1)
max_output = torch.max(logits_probabilities, axis=1)
probabilities = max_output.values.item()
sentiments = max_output.indices.item()
print(sentiments)

0


In [29]:
print('These are the available domains')
for i in model_groups:
    print(i)

These are the available domains
books_de_music
books_de_dvd
books_jp_music
books_jp_dvd
books_fr_music
books_fr_dvd
music_de_books
music_de_dvd
music_jp_books
music_jp_dvd
music_fr_books
music_fr_dvd
dvd_de_books
dvd_de_music
dvd_jp_books
dvd_jp_music
dvd_fr_books
dvd_fr_music
